In [1]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

In [2]:
ratings=spark.read.csv("ratings.csv",header=True)

In [3]:
ratings.columns

['userId', 'movieId', 'rating', 'timestamp']

In [4]:
ratings.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      2|   3.5|1112486027|
|     1|     29|   3.5|1112484676|
|     1|     32|   3.5|1112484819|
|     1|     47|   3.5|1112484727|
|     1|     50|   3.5|1112484580|
|     1|    112|   3.5|1094785740|
|     1|    151|   4.0|1094785734|
|     1|    223|   4.0|1112485573|
|     1|    253|   4.0|1112484940|
|     1|    260|   4.0|1112484826|
|     1|    293|   4.0|1112484703|
|     1|    296|   4.0|1112484767|
|     1|    318|   4.0|1112484798|
|     1|    337|   3.5|1094785709|
|     1|    367|   3.5|1112485980|
|     1|    541|   4.0|1112484603|
|     1|    589|   3.5|1112485557|
|     1|    593|   3.5|1112484661|
|     1|    653|   3.0|1094785691|
|     1|    919|   3.5|1094785621|
+------+-------+------+----------+
only showing top 20 rows



In [6]:
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  99.46% empty.


In [8]:
# Group data by userId, count ratings
ratings.groupBy("userId").count().show()

+------+-----+
|userId|count|
+------+-----+
|   296|   25|
|   467|   30|
|   675|  187|
|   691|   35|
|   829|  387|
|  1090|   74|
|  1159|  235|
|  1436|  234|
|  1512|   68|
|  1572|   64|
|  2069|   45|
|  2088|   87|
|  2136|  201|
|  2162|  100|
|  2294|   21|
|  2904|   23|
|  3210|  452|
|  3414|   29|
|  3606|   66|
|  3959|   24|
+------+-----+
only showing top 20 rows



In [9]:
from pyspark.sql.functions import min,avg

In [10]:
# Min num ratings for movies
print("Movie with the fewest ratings: ")
ratings.groupBy("movieId").count().select(min("count")).show()

Movie with the fewest ratings: 
+----------+
|min(count)|
+----------+
|         1|
+----------+



In [11]:
# Avg num ratings per movie
print("Avg num ratings per movie: ")
ratings.groupBy("movieId").count().select(avg("count")).show()

Avg num ratings per movie: 
+-----------------+
|       avg(count)|
+-----------------+
|747.8411232425965|
+-----------------+



In [12]:
# Min num ratings for user
print("User with the fewest ratings: ")
ratings.groupBy("userId").count().select(min("count")).show()

User with the fewest ratings: 
+----------+
|min(count)|
+----------+
|        20|
+----------+



In [13]:
# Avg num ratings per users
print("Avg num ratings per user: ")
ratings.groupBy("userId").count().select(avg("count")).show()

Avg num ratings per user: 
+-----------------+
|       avg(count)|
+-----------------+
|144.4135299257002|
+-----------------+



In [14]:
# Use .printSchema() to see the datatypes of the ratings dataset
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [15]:
# Tell Spark to convert the columns to the proper data types
ratings = ratings.select(ratings.userId.cast("integer"), ratings.movieId.cast("integer"), ratings.rating.cast("double"))

# Call .printSchema() again to confirm the columns are now in the correct format
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [16]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [17]:
# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

In [18]:
# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False)

# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

In [21]:
# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.maxIter, [5, 50, 100, 200]) \
            .addGrid(als.regParam, [0.01, 0.05, 0.1, 0.15]) \
            .build()
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  64


In [22]:
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 

In [23]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)

CrossValidator_ee1b4eed298e


In [24]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

Py4JJavaError: An error occurred while calling o196.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 35.0 failed 1 times, most recent failure: Lost task 3.0 in stage 35.0 (TID 1865, localhost, executor driver): java.lang.NoSuchMethodError: sun.nio.ch.DirectBuffer.cleaner()Lsun/misc/Cleaner;
	at org.apache.spark.storage.StorageUtils$.cleanDirectBuffer(StorageUtils.scala:212)
	at org.apache.spark.storage.StorageUtils$.dispose(StorageUtils.scala:207)
	at org.apache.spark.storage.StorageUtils.dispose(StorageUtils.scala)
	at org.apache.spark.io.NioBufferedFileInputStream.close(NioBufferedFileInputStream.java:130)
	at java.base/java.io.FilterInputStream.close(FilterInputStream.java:180)
	at org.apache.spark.io.ReadAheadInputStream.close(ReadAheadInputStream.java:400)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillReader.close(UnsafeSorterSpillReader.java:152)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpill

In [ ]:
#Extract best model from the cv model above
best_model = model.bestModel